<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IMPORTS" data-toc-modified-id="IMPORTS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IMPORTS</a></span></li><li><span><a href="#READ-IN" data-toc-modified-id="READ-IN-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>READ-IN</a></span></li><li><span><a href="#DATA-QUALITY" data-toc-modified-id="DATA-QUALITY-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DATA QUALITY</a></span></li><li><span><a href="#PRIMARY-TRANSFORMATIONS" data-toc-modified-id="PRIMARY-TRANSFORMATIONS-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>PRIMARY TRANSFORMATIONS</a></span><ul class="toc-item"><li><span><a href="#ADD-FEATURE-MONTH" data-toc-modified-id="ADD-FEATURE-MONTH-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>ADD FEATURE MONTH</a></span></li><li><span><a href="#ADD-FEATURE-DAYOFWEEK" data-toc-modified-id="ADD-FEATURE-DAYOFWEEK-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>ADD FEATURE DAYOFWEEK</a></span></li><li><span><a href="#ADD-FEATURE-TIME-OF-DAY" data-toc-modified-id="ADD-FEATURE-TIME-OF-DAY-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>ADD FEATURE TIME OF DAY</a></span></li><li><span><a href="#ADD-FEATURE-SCHOOL-IN-SESSION-/-BANK-HOLIDAY" data-toc-modified-id="ADD-FEATURE-SCHOOL-IN-SESSION-/-BANK-HOLIDAY-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>ADD FEATURE SCHOOL IN SESSION / BANK HOLIDAY</a></span></li></ul></li><li><span><a href="#ADD-WEATHER-DATA" data-toc-modified-id="ADD-WEATHER-DATA-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>ADD WEATHER DATA</a></span></li><li><span><a href="#DROP-COLUMNS" data-toc-modified-id="DROP-COLUMNS-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>DROP COLUMNS</a></span></li><li><span><a href="#SET-DATATYPES" data-toc-modified-id="SET-DATATYPES-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>SET DATATYPES</a></span></li><li><span><a href="#SAVE-THIS-DATAFRAME" data-toc-modified-id="SAVE-THIS-DATAFRAME-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>SAVE THIS DATAFRAME</a></span></li><li><span><a href="#TRAIN-—-VALIDATION-—-TEST" data-toc-modified-id="TRAIN-—-VALIDATION-—-TEST-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>TRAIN — VALIDATION — TEST</a></span></li></ul></div>

# IMPORTS

In [2]:
# set the display aspect ratio

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:80% !important;}</style>'))

In [1]:
# import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as papr
from matplotlib import style
import gc

In [4]:
# Display plots in the notebook

%matplotlib inline

In [5]:
# load custom function for assessing dataframes succinctly

def assess_df(df):
  
    feature_types = {df[df.columns[x]].dtypes.name for x in range(len(df.columns))}
    
  	# print datatypes
  
    print("Datatypes:\n")
    print(df.dtypes)
    print('\n\n')
    
    # print number rows and cols
    print('*' * 50)
    print('ROWS, COLUMNS, NULLS\n')
    print(df.shape[0], "rows in the dataframe.")
    print(df.shape[1], "columns in the dataframe.\n")
    print(df.isnull().sum().sum(), "null values in the dataframe.")
    
    # check duplicate rows & col; print if present
    
    duplicate_rows = df.duplicated(keep=False).sum()
    duplicate_cols = df.index.T.duplicated(keep=False).sum()
    
    print(duplicate_rows, "duplicate rows")
    print(duplicate_cols, "duplicate columns")
    
    if duplicate_rows > 0 or duplicate_cols > 0:
      print(df.loc[df.duplicated()])
	
    print('\n')
    
    # check for constant columns
    
    numeric_cols = list(df.select_dtypes(include=[np.number]).columns.values)
    categorical_cols = list(df.select_dtypes('category').columns.values)
    
    standard_diffs = [df.describe().loc['std', x] for x in numeric_cols]
    
    constant_categorical_column=False
    if 'category' in feature_types:
        for column in categorical_cols:
            if df[column].describe().index.unique == 1:
                constant_categorical_column = True
            
            
    if 0 in standard_diffs:
        print("Constant numeric columns: TRUE")
    else:
        print("Constant numeric columns: FALSE")
  
    if 'category' in feature_types:    
        if constant_categorical_column == True:
            print("Constant categorical columns: TRUE")
        else:
            print("Constant categorical columns: FALSE")

            
    
    print('\n\n')

    
    # feature stats
    print('*' * 50)
    print('DESCRIPTION\n')
    print(df.describe().T)
    print('\n\n')
    print('*' * 50)

    
    # feature stats: categorical
    
    if 'category' in feature_types:
        print('CATEGORICAL DESCRIPTION\n')
        print('\n')
        print(df.select_dtypes(['category']).describe().T)
        print('\n\n')
        print('*' * 50)
    
    # print feature cardinalities
    
    print("FEATURE CARDINALITIES\n")
    column_names = list(df.columns.values)

    print('{0:45}  {1}'.format("Feature", "Distinct Values"))
    print('{0:45}  {1}'.format("-------", "--------------- \n"))

    for c in column_names:
        print('{0:45}  {1}'.format(c, str(len(df[c].unique()))))
    
    print('\n')
    print('*' * 50)
    print('MEMORY\n')
    
    
    print(df.info(memory_usage='deep'))
    print('\n')
    print(df.memory_usage(deep=True))
    print('\n')
    print('*' * 50)
    print('HEAD\n')
    print(df.head(10))
    print('\n')
    print('*' * 50)
    print('TAIL\n')
    print(df.tail(10))

# READ-IN

In [9]:
df = pd.read_feather('/tmp/ssh_mount/data/dataframes/L145/df_L145_leavetimes.feather')

In [8]:
weather = pd.read_feather('/Users/davidodwyer/Documents/studyCS/Semester_3/data/dataframes/L145/weather.feather')

# DATA QUALITY

In [9]:
assess_df(df)

Datatypes:

DAYOFSERVICE       datetime64[ns]
TRIPID                      int32
PROGRNUMBER              category
STOPPOINTID              category
PLANNEDTIME_ARR             int32
PLANNEDTIME_DEP             int32
ACTUALTIME_ARR              int32
ACTUALTIME_DEP              int32
VEHICLEID                category
dtype: object



**************************************************
ROWS, COLUMNS, NULLS

4027897 rows in the dataframe.
9 columns in the dataframe.

0 null values in the dataframe.
0 duplicate rows
0 duplicate columns


Constant numeric columns: FALSE
Constant categorical columns: FALSE



**************************************************
DESCRIPTION

                     count          mean            std        min        25%  \
TRIPID           4027897.0  7.144615e+06  733250.132918  5955476.0  6592921.0   
PLANNEDTIME_ARR  4027897.0  5.391299e+04   16893.384067    22200.0    39107.0   
PLANNEDTIME_DEP  4027897.0  5.391299e+04   16893.384067    22200.0    39107.0   
AC

In [10]:
df.TRIPID = df.TRIPID.astype('category')

In [11]:
df.head(2)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334
1,2018-02-18,6263611,66,2964,57113,57113,57459,57483,2868334


# PRIMARY TRANSFORMATIONS

## ADD FEATURE MONTH

In [12]:
# add month

df['MONTH'] = df.DAYOFSERVICE.dt.month

## ADD FEATURE DAYOFWEEK

In [13]:
# add day of week

df['DAYOFWEEK'] = df.DAYOFSERVICE.dt.dayofweek

## ADD FEATURE TIME OF DAY

In [14]:
# create bins and labels for time of day feature

max_actual_dept = df.ACTUALTIME_ARR.max()
bins = []
x = 0
while x <= max_actual_dept:
    bins.append(x)
    x+=1800
bins.append(x)
len(bins)

52

In [15]:
labels = [
    '00:00 - 00:30',
    '00:30 - 01:00',
    '01:00 - 01:30',
    '01:30 - 02:00',
    '02:00 - 02:30',
    '02:30 - 03:00',
    '03:00 - 03:30',
    '03:30 - 04:00',
    '04:00 - 04:30',
    '04:30 - 05:00',
    '05:00 - 05:30',
    '05:30 - 06:00',
    '06:00 - 06:30',
    '06:30 - 07:00',
    '07:00 - 07:30',
    '07:30 - 08:00',
    '08:00 - 08:30',
    '08:30 - 09:00',
    '09:00 - 09:30',
    '09:30 - 10:00',
    '10:00 - 10:30',
    '10:30 - 11:00',
    '11:00 - 11:30',
    '11:30 - 12:00',
    '12:00 - 12:30',
    '12:30 - 13:00',
    '13:00 - 13:30',
    '13:30 - 14:00',
    '14:00 - 14:30',
    '14:30 - 15:00',
    '15:00 - 15:30',
    '15:30 - 16:00',
    '16:00 - 16:30',
    '16:30 - 17:00',
    '17:00 - 17:30',
    '17:30 - 18:00',
    '18:00 - 18:30',
    '18:30 - 19:00',
    '19:00 - 19:30',
    '19:30 - 20:00',
    '20:00 - 20:30',
    '20:30 - 21:00',
    '21:00 - 21:30',
    '21:30 - 22:00',
    '22:00 - 22:30',
    '22:30 - 23:00',
    '23:00 - 23:30',
    '23:30 - 24:00',
    '+24:00 - 00:30',
    '+00:30 - 01:00',
    '+01:00 - 01:30'
]

In [16]:
# create time of day period feature

df['TIME_PERIOD_ARRIVAL'] = pd.cut(df.ACTUALTIME_DEP, bins=bins, labels=labels)

In [17]:
df.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334,2,6,15:00 - 15:30
1,2018-02-18,6263611,66,2964,57113,57113,57459,57483,2868334,2,6,15:30 - 16:00
2,2018-02-18,6263611,67,4175,57174,57174,57611,57635,2868334,2,6,16:00 - 16:30
3,2018-02-18,6263611,32,2014,55200,55200,55277,55286,2868334,2,6,15:00 - 15:30
4,2018-02-18,6263611,33,4636,55238,55238,55315,55328,2868334,2,6,15:00 - 15:30


Check that the binning was done correctly

In [18]:
# test

df.loc[(df.TIME_PERIOD_ARRIVAL >'23:00 - 23:30')].head().iloc[0,:]

DAYOFSERVICE           2018-04-13 00:00:00
TRIPID                             6619984
PROGRNUMBER                             69
STOPPOINTID                           1443
PLANNEDTIME_ARR                      83738
PLANNEDTIME_DEP                      83738
ACTUALTIME_ARR                       84818
ACTUALTIME_DEP                       84818
VEHICLEID                          2172277
MONTH                                    4
DAYOFWEEK                                4
TIME_PERIOD_ARRIVAL          23:30 - 24:00
Name: 16631, dtype: object

In [19]:
df.loc[(df.TIME_PERIOD_ARRIVAL >'23:00 - 23:30')].head().iloc[0,:]['ACTUALTIME_ARR'] / 60 / 60

23.560555555555556

In [20]:
# test

df.loc[(df.TIME_PERIOD_ARRIVAL >'13:00 - 13:30')].head().iloc[0,:]

DAYOFSERVICE           2018-02-18 00:00:00
TRIPID                             6263611
PROGRNUMBER                             31
STOPPOINTID                           2013
PLANNEDTIME_ARR                      55128
PLANNEDTIME_DEP                      55128
ACTUALTIME_ARR                       55231
ACTUALTIME_DEP                       55240
VEHICLEID                          2868334
MONTH                                    2
DAYOFWEEK                                6
TIME_PERIOD_ARRIVAL          15:00 - 15:30
Name: 0, dtype: object

In [21]:
df.loc[(df.TIME_PERIOD_ARRIVAL >'13:00 - 13:30')].head().iloc[0,:]['ACTUALTIME_ARR'] / 60 / 60

15.341944444444444

## ADD FEATURE SCHOOL IN SESSION / BANK HOLIDAY

In [23]:
holiday_dates = [
    '1/1/18',
    '2/1/18',
    '3/1/18',
    '4/1/18',
    '5/1/18',
    '6/1/18',
    '7/1/18',
    '15/2/18',
    '16/2/18',
    '17/3/18',
    '23/3/18',
    '24/3/18',
    '25/3/18',
    '26/3/18',
    '27/3/18',
    '28/3/18',
    '29/3/18',
    '30/3/18',
    '31/3/18',
    '1/4/18',
    '2/4/18',
    '3/4/18',
    '4/4/18',
    '5/4/18',
    '6/4/18',
    '7/4/18',
    '8/4/18',
    '1/5/18',
    '7/5/18',
    '4/6/18',
    '6/8/18',
    '29/10/18',
    '30/10/18',
    '31/10/18',
    '1/11/18',
    '2/11/18',
    '21/12/18',
    '22/12/18',
    '23/12/18',
    '24/12/18',
    '25/12/18',
    '26/12/18',
    '27/12/18',
    '28/12/18',
    '29/12/18',
    '30/12/18',
    '31/12/18'
]

In [24]:
school_bank_dates = pd.to_datetime(holiday_dates)

In [25]:
school_bank_dates

DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-02-15',
               '2018-02-16', '2018-03-17', '2018-03-23', '2018-03-24',
               '2018-03-25', '2018-03-26', '2018-03-27', '2018-03-28',
               '2018-03-29', '2018-03-30', '2018-03-31', '2018-01-04',
               '2018-02-04', '2018-03-04', '2018-04-04', '2018-05-04',
               '2018-06-04', '2018-07-04', '2018-08-04', '2018-01-05',
               '2018-07-05', '2018-04-06', '2018-06-08', '2018-10-29',
               '2018-10-30', '2018-10-31', '2018-01-11', '2018-02-11',
               '2018-12-21', '2018-12-22', '2018-12-23', '2018-12-24',
               '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28',
               '2018-12-29', '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns]', freq=None)

In [26]:
len(school_bank_dates)

47

In [27]:
# start and end dates of primary school summer holidays

start = pd.to_datetime('28/6/2018')
end = pd.to_datetime('1/Sep/2018')

print(start)
print(end)


2018-06-28 00:00:00
2018-09-01 00:00:00


In [28]:
summer_break = pd.DatetimeIndex(start=start, end=end, freq='d')

/Users/davidodwyer/anaconda3/envs/analytics/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  """Entry point for launching an IPython kernel.


In [29]:
# create datetime index of summer dates

summer_break

DatetimeIndex(['2018-06-28', '2018-06-29', '2018-06-30', '2018-07-01',
               '2018-07-02', '2018-07-03', '2018-07-04', '2018-07-05',
               '2018-07-06', '2018-07-07', '2018-07-08', '2018-07-09',
               '2018-07-10', '2018-07-11', '2018-07-12', '2018-07-13',
               '2018-07-14', '2018-07-15', '2018-07-16', '2018-07-17',
               '2018-07-18', '2018-07-19', '2018-07-20', '2018-07-21',
               '2018-07-22', '2018-07-23', '2018-07-24', '2018-07-25',
               '2018-07-26', '2018-07-27', '2018-07-28', '2018-07-29',
               '2018-07-30', '2018-07-31', '2018-08-01', '2018-08-02',
               '2018-08-03', '2018-08-04', '2018-08-05', '2018-08-06',
               '2018-08-07', '2018-08-08', '2018-08-09', '2018-08-10',
               '2018-08-11', '2018-08-12', '2018-08-13', '2018-08-14',
               '2018-08-15', '2018-08-16', '2018-08-17', '2018-08-18',
               '2018-08-19', '2018-08-20', '2018-08-21', '2018-08-22',
      

In [30]:
len(summer_break)

66

In [31]:
# join the two date sets

school_off = summer_break.append(school_bank_dates)

In [32]:
school_off

DatetimeIndex(['2018-06-28', '2018-06-29', '2018-06-30', '2018-07-01',
               '2018-07-02', '2018-07-03', '2018-07-04', '2018-07-05',
               '2018-07-06', '2018-07-07',
               ...
               '2018-12-22', '2018-12-23', '2018-12-24', '2018-12-25',
               '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29',
               '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns]', length=113, freq=None)

In [33]:
len(school_off)

113

In [34]:
# initialise new feature

df['SCHOOL_OFF'] = '0'

Find rows where it is a holday, impute 1 value for these rows

In [35]:
school_off_indices = []

for item in school_off:
    temp_list = df.loc[df.DAYOFSERVICE == item].index.to_list()
    for i in temp_list:
        school_off_indices.append(i)

In [36]:
df.iloc[school_off_indices, 12] = '1' 

In [37]:
df.SCHOOL_OFF.value_counts()

0    2900471
1    1127426
Name: SCHOOL_OFF, dtype: int64

# ADD WEATHER DATA

In [38]:
df.head(2)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334,2,6,15:00 - 15:30,0
1,2018-02-18,6263611,66,2964,57113,57113,57459,57483,2868334,2,6,15:30 - 16:00,0


In [39]:
weather.head()

,date,rain,temp
0,2018-01-01 00:00:00,0.033325,4.167969
1,2018-01-01 01:00:00,0.033325,4.332031
2,2018-01-01 02:00:00,0.000000,4.402344
3,2018-01-01 03:00:00,0.000000,4.667969
4,2018-01-01 04:00:00,0.000000,4.902344


In [40]:
assess_df(weather)

Datatypes:

date    datetime64[ns]
rain           float32
temp           float32
dtype: object



**************************************************
ROWS, COLUMNS, NULLS

8737 rows in the dataframe.
3 columns in the dataframe.

0 null values in the dataframe.
0 duplicate rows
0 duplicate columns


Constant numeric columns: FALSE



**************************************************
DESCRIPTION

       count       mean       std       min       25%       50%      75%  \
rain  8737.0   0.077775  0.313557  0.000000  0.000000  0.000000   0.0000   
temp  8737.0  10.095499  5.583059 -4.867188  6.066406  9.734375  14.0625   

            max  
rain   6.066406  
temp  26.968750  



**************************************************
FEATURE CARDINALITIES

Feature                                        Distinct Values
-------                                        --------------- 

date                                           8737
rain                                           130
temp       

In [41]:
# create time and hour columns for ease of joining with df

dates, hours = zip(*[(d.date(), d.time()) for d in weather['date']])

weather = weather.assign(dates=dates, hours=hours)

In [42]:
# check dtypes

weather.dtypes

date     datetime64[ns]
rain            float32
temp            float32
dates            object
hours            object
dtype: object

In [43]:
# check type

type(weather.ix[1, 'hours'])

/Users/davidodwyer/anaconda3/envs/analytics/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


datetime.time

In [44]:
# change type

weather.hours = weather.hours.astype('str')

In [45]:
# define mapping

replacements_hours = {
    '00:00:00' : '0',
    '01:00:00' : '1',
    '02:00:00' : '2',
    '03:00:00' : '3',
    '04:00:00' : '4',
    '05:00:00' : '5',
    '06:00:00' : '6',
    '07:00:00' : '7',
    '08:00:00' : '8',
    '09:00:00' : '9',
    '10:00:00' : '10',
    '11:00:00' : '11',
    '12:00:00' : '12',
    '13:00:00' : '13',
    '14:00:00' : '14',
    '15:00:00' : '15',
    '16:00:00' : '16',
    '17:00:00' : '17',
    '18:00:00' : '18',
    '19:00:00' : '19',
    '20:00:00' : '20',
    '21:00:00' : '21',
    '22:00:00' : '22',
    '23:00:00' : '23'
}

In [46]:
# perform mapping

weather.hours = weather.hours.map(replacements_hours)

In [47]:
#inpect

weather.head(25)

,date,rain,temp,dates,hours
0,2018-01-01 00:00:00,0.033325,4.167969,2018-01-01,0
1,2018-01-01 01:00:00,0.033325,4.332031,2018-01-01,1
2,2018-01-01 02:00:00,0.000000,4.402344,2018-01-01,2
3,2018-01-01 03:00:00,0.000000,4.667969,2018-01-01,3
4,2018-01-01 04:00:00,0.000000,4.902344,2018-01-01,4
5,2018-01-01 05:00:00,0.000000,4.800781,2018-01-01,5
6,2018-01-01 06:00:00,0.000000,4.531250,2018-01-01,6
7,2018-01-01 07:00:00,0.000000,4.167969,2018-01-01,7
8,2018-01-01 08:00:00,0.000000,3.933594,2018-01-01,8
9,2018-01-01 09:00:00,0.000000,4.402344,2018-01-01,9


In [48]:
# change to datetime

weather.dates = weather.dates.astype('datetime64[ns]')

In [49]:
# check dtype

weather.dates.dtype

dtype('<M8[ns]')

In [50]:
weather.isna().sum()

date     0
rain     0
temp     0
dates    0
hours    0
dtype: int64

In [51]:
weather.loc[weather.date == pd.to_datetime('31/12/18')]

,date,rain,temp,dates,hours
8736,2018-12-31,0.033325,8.960938,2018-12-31,0


<br>

Temporarily Amend DF for Mapping Weather to Bus Data

<br>

In [52]:
df.head(1)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334,2,6,15:00 - 15:30,0


In [53]:
type(df.ix[0, 'TIME_PERIOD_ARRIVAL'])

/Users/davidodwyer/anaconda3/envs/analytics/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


str

In [54]:
replacement_time_period = {
    '00:00 - 00:30' : '0',
    '00:30 - 01:00' : '0',
    '01:00 - 01:30' : '1',
    '01:30 - 02:00' : '1',
    '02:00 - 02:30' : '2',
    '02:30 - 03:00' : '2',
    '03:00 - 03:30' : '3',
    '03:30 - 04:00' : '3',
    '04:00 - 04:30' : '4',
    '04:30 - 05:00' : '4',
    '05:00 - 05:30' : '5',
    '05:30 - 06:00' : '5',
    '06:00 - 06:30' : '6',
    '06:30 - 07:00' : '6',
    '07:00 - 07:30' : '7',
    '07:30 - 08:00' : '7',
    '08:00 - 08:30' : '8',
    '08:30 - 09:00' : '8',
    '09:00 - 09:30' : '9',
    '09:30 - 10:00' : '9',
    '10:00 - 10:30' : '10',
    '10:30 - 11:00' : '10',
    '11:00 - 11:30' : '11',
    '11:30 - 12:00' : '11',
    '12:00 - 12:30' : '12',
    '12:30 - 13:00' : '12',
    '13:00 - 13:30' : '13',
    '13:30 - 14:00' : '13',
    '14:00 - 14:30' : '14',
    '14:30 - 15:00' : '14',
    '15:00 - 15:30' : '15',
    '15:30 - 16:00' : '15',
    '16:00 - 16:30' : '16',
    '16:30 - 17:00' : '16',
    '17:00 - 17:30' : '17',
    '17:30 - 18:00' : '17',
    '18:00 - 18:30' : '18',
    '18:30 - 19:00' : '18',
    '19:00 - 19:30' : '19',
    '19:30 - 20:00' : '19',
    '20:00 - 20:30' : '20',
    '20:30 - 21:00' : '20',
    '21:00 - 21:30' : '21',
    '21:30 - 22:00' : '21',
    '22:00 - 22:30' : '22',
    '22:30 - 23:00' : '22',
    '23:00 - 23:30' : '23',
    '23:30 - 24:00' : '23',
    '+24:00 - 00:30' : '0',
    '+00:30 - 01:00' : '0',
    '+01:00 - 01:30' : '1'
}

In [55]:
df['temp'] = df['TIME_PERIOD_ARRIVAL']

In [56]:
df.head(1)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF,temp
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334,2,6,15:00 - 15:30,0,15:00 - 15:30


In [57]:
df.temp = df.temp.map(replacement_time_period)

In [58]:
df.head(2)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF,temp
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334,2,6,15:00 - 15:30,0,15
1,2018-02-18,6263611,66,2964,57113,57113,57459,57483,2868334,2,6,15:30 - 16:00,0,15


In [59]:
df.rename({'temp': 'hours'}, axis=1, inplace=True)

In [60]:
df.head(2)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF,hours
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334,2,6,15:00 - 15:30,0,15
1,2018-02-18,6263611,66,2964,57113,57113,57459,57483,2868334,2,6,15:30 - 16:00,0,15


In [61]:
df_merged = pd.merge(df, weather, how='left', left_on=['DAYOFSERVICE', 'hours'], right_on=['dates', 'hours'])

In [62]:
df_merged.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF,hours,date,rain,temp,dates
0,2018-02-18,6263611,31,2013,55128,55128,55231,55240,2868334,2,6,15:00 - 15:30,0,15,2018-02-18 15:00:00,0.000000,10.000000,2018-02-18
1,2018-02-18,6263611,66,2964,57113,57113,57459,57483,2868334,2,6,15:30 - 16:00,0,15,2018-02-18 15:00:00,0.000000,10.000000,2018-02-18
2,2018-02-18,6263611,67,4175,57174,57174,57611,57635,2868334,2,6,16:00 - 16:30,0,16,2018-02-18 16:00:00,0.199951,10.070312,2018-02-18
3,2018-02-18,6263611,32,2014,55200,55200,55277,55286,2868334,2,6,15:00 - 15:30,0,15,2018-02-18 15:00:00,0.000000,10.000000,2018-02-18
4,2018-02-18,6263611,33,4636,55238,55238,55315,55328,2868334,2,6,15:00 - 15:30,0,15,2018-02-18 15:00:00,0.000000,10.000000,2018-02-18


In [63]:
# pd.options.display.max_rows = 6500

df_merged.loc[df_merged.rain.isna()]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF,hours,date,rain,temp,dates
56399,2018-12-31,8587286,47,3135,62593,62593,63415,63433,2172238,12,0,17:30 - 18:00,1,17,NaT,NaN,NaN,NaT
56400,2018-12-31,8587286,48,3136,62659,62659,63495,63511,2172238,12,0,17:30 - 18:00,1,17,NaT,NaN,NaN,NaT
56401,2018-12-31,8587286,26,2009,61485,61485,61997,62032,2172238,12,0,17:00 - 17:30,1,17,NaT,NaN,NaN,NaT
56402,2018-12-31,8587286,49,3138,62702,62702,63560,63577,2172238,12,0,17:30 - 18:00,1,17,NaT,NaN,NaN,NaT
56403,2018-12-31,8587286,27,2010,61571,61571,62141,62156,2172238,12,0,17:00 - 17:30,1,17,NaT,NaN,NaN,NaT
56404,2018-12-31,8587286,50,3139,62783,62783,63647,63673,2172238,12,0,17:30 - 18:00,1,17,NaT,NaN,NaN,NaT
56405,2018-12-31,8587286,51,4124,62809,62809,63714,63729,2172238,12,0,17:30 - 18:00,1,17,NaT,NaN,NaN,NaT
56406,2018-12-31,8587286,29,7353,61670,61670,62257,62276,2172238,12,0,17:00 - 17:30,1,17,NaT,NaN,NaN,NaT
56407,2018-12-31,8587286,52,4125,62863,62863,63778,63778,2172238,12,0,17:30 - 18:00,1,17,NaT,NaN,NaN,NaT
56408,2018-12-31,8587286,30,4571,61768,61768,62371,62408,2172238,12,0,17:00 - 17:30,1,17,NaT,NaN,NaN,NaT


# DROP COLUMNS

In [64]:
df_merged.drop([
    'DAYOFSERVICE',
    'TRIPID',
    'PROGRNUMBER',
    'VEHICLEID',
    'date',
    'dates'
], axis=1, inplace=True)

In [65]:
df_merged.drop('hours', axis=1, inplace=True)

In [66]:
df_merged.drop([
    'PLANNEDTIME_DEP',
    'ACTUALTIME_DEP'
], axis=1, inplace=True)

In [67]:
df_merged.head(3)

,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,SCHOOL_OFF,rain,temp
0,2013,55128,55231,2,6,15:00 - 15:30,0,0.000000,10.000000
1,2964,57113,57459,2,6,15:30 - 16:00,0,0.000000,10.000000
2,4175,57174,57611,2,6,16:00 - 16:30,0,0.199951,10.070312


In [68]:
df_merged.iloc[0,7]

0.0

In [69]:
# reorder rows

df_merged = df_merged[[
    'STOPPOINTID',
    'MONTH',
    'DAYOFWEEK',
    'TIME_PERIOD_ARRIVAL',
    'PLANNEDTIME_ARR',
    'SCHOOL_OFF',
    'rain',
    'temp',
    'ACTUALTIME_ARR'
]]

In [70]:
df_merged.head()

,STOPPOINTID,MONTH,DAYOFWEEK,TIME_PERIOD_ARRIVAL,PLANNEDTIME_ARR,SCHOOL_OFF,rain,temp,ACTUALTIME_ARR
0,2013,2,6,15:00 - 15:30,55128,0,0.000000,10.000000,55231
1,2964,2,6,15:30 - 16:00,57113,0,0.000000,10.000000,57459
2,4175,2,6,16:00 - 16:30,57174,0,0.199951,10.070312,57611
3,2014,2,6,15:00 - 15:30,55200,0,0.000000,10.000000,55277
4,4636,2,6,15:00 - 15:30,55238,0,0.000000,10.000000,55315


# SET DATATYPES

In [71]:
df_merged.dtypes

STOPPOINTID            category
MONTH                     int64
DAYOFWEEK                 int64
TIME_PERIOD_ARRIVAL    category
PLANNEDTIME_ARR           int32
SCHOOL_OFF               object
rain                    float32
temp                    float32
ACTUALTIME_ARR            int32
dtype: object

In [72]:
# change some dtypes

df_merged = df_merged.astype({
    'MONTH' : 'category',
    'DAYOFWEEK' : 'category',
    'SCHOOL_OFF' : 'category',    
})

In [73]:
# rename cols

df_merged.rename(columns={
    'STOPPOINTID': 'STOP_ID',
    'TIME_PERIOD_ARRIVAL' : 'ARRIVAL_INTERVAL',
    'rain' : 'RAIN',
    'temp' : 'TEMP'
}, inplace=True)


In [74]:
df_merged.head()

,STOP_ID,MONTH,DAYOFWEEK,ARRIVAL_INTERVAL,PLANNEDTIME_ARR,SCHOOL_OFF,RAIN,TEMP,ACTUALTIME_ARR
0,2013,2,6,15:00 - 15:30,55128,0,0.000000,10.000000,55231
1,2964,2,6,15:30 - 16:00,57113,0,0.000000,10.000000,57459
2,4175,2,6,16:00 - 16:30,57174,0,0.199951,10.070312,57611
3,2014,2,6,15:00 - 15:30,55200,0,0.000000,10.000000,55277
4,4636,2,6,15:00 - 15:30,55238,0,0.000000,10.000000,55315


In [75]:
del df

In [76]:
assess_df(df_merged)

Datatypes:

STOP_ID             category
MONTH               category
DAYOFWEEK           category
ARRIVAL_INTERVAL    category
PLANNEDTIME_ARR        int32
SCHOOL_OFF          category
RAIN                 float32
TEMP                 float32
ACTUALTIME_ARR         int32
dtype: object



**************************************************
ROWS, COLUMNS, NULLS

4027897 rows in the dataframe.
9 columns in the dataframe.

12594 null values in the dataframe.
16 duplicate rows
0 duplicate columns
        STOP_ID MONTH DAYOFWEEK ARRIVAL_INTERVAL  PLANNEDTIME_ARR SCHOOL_OFF  \
383367     2964     4         0    08:30 - 09:00            30040          0   
393003     2069     4         0    08:00 - 08:30            29696          0   
397482     4182     3         3    20:30 - 21:00            74813          0   
492654     2014     4         0    08:30 - 09:00            31489          0   
509371     4407     3         3    20:30 - 21:00            74177          0   
1076513     773     8  

In [77]:
# check for nulls

for column in df_merged.columns:
    print(column, df_merged[column].isna().sum())

STOP_ID 0
MONTH 0
DAYOFWEEK 0
ARRIVAL_INTERVAL 0
PLANNEDTIME_ARR 0
SCHOOL_OFF 0
RAIN 6297
TEMP 6297
ACTUALTIME_ARR 0


<br>

There is no weather data for Dec 31

<br>

In [78]:
# drop these rows

df_merged.dropna(inplace=True)

In [79]:
df_merged.isna().sum()

STOP_ID             0
MONTH               0
DAYOFWEEK           0
ARRIVAL_INTERVAL    0
PLANNEDTIME_ARR     0
SCHOOL_OFF          0
RAIN                0
TEMP                0
ACTUALTIME_ARR      0
dtype: int64

In [80]:
df_merged.loc[df_merged.duplicated(keep=False)]

,STOP_ID,MONTH,DAYOFWEEK,ARRIVAL_INTERVAL,PLANNEDTIME_ARR,SCHOOL_OFF,RAIN,TEMP,ACTUALTIME_ARR
383318,2964,4,0,08:30 - 09:00,30040,0,0.0,9.429688,31106
383367,2964,4,0,08:30 - 09:00,30040,0,0.0,9.429688,31106
392942,2069,4,0,08:00 - 08:30,29696,0,0.0,9.429688,30141
393003,2069,4,0,08:00 - 08:30,29696,0,0.0,9.429688,30141
397173,4182,3,3,20:30 - 21:00,74813,0,0.0,8.468750,74897
397482,4182,3,3,20:30 - 21:00,74813,0,0.0,8.468750,74897
492538,2014,4,0,08:30 - 09:00,31489,0,0.0,9.429688,32083
492654,2014,4,0,08:30 - 09:00,31489,0,0.0,9.429688,32083
509031,4407,3,3,20:30 - 21:00,74177,0,0.0,8.468750,74724
509371,4407,3,3,20:30 - 21:00,74177,0,0.0,8.468750,74724


In [81]:
df_merged.drop_duplicates(inplace=True)

# SAVE THIS DATAFRAME

In [82]:
df_merged.reset_index(inplace=True, drop=True)
df_merged.to_feather('/Users/davidodwyer/Documents/studyCS/Semester_3/data/dataframes/L145/145ABT.feather')